In [ ]:
# | default_exp _cli

In [ ]:
# | export

import asyncio
import multiprocessing
from typing import *

import typer

from fastkafka import _cli_docs, _cli_testing
from fastkafka._components.logger import get_logger
from fastkafka._server import run_fastkafka_server

/home/kumaran/.local/lib/python3.11/site-packages/pydantic/_internal/_config.py:257: UserWarning: Valid config keys have changed in V2:
* 'json_encoders' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
import os
import platform
import time

from typer.testing import CliRunner

from fastkafka._components.logger import suppress_timestamps
from fastkafka._components.test_dependencies import generate_app_in_tmp
from fastkafka._server import terminate_asyncio_process
from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__, level=20)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")

In [ ]:
# | export


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(
        multiprocessing.cpu_count(),
        help="Number of FastKafka instances to run, defaults to number of CPU cores.",
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
    kafka_broker: str = typer.Option(
        "localhost",
        help="kafka_broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastaKafka class.",
    ),
) -> None:
    """
    Runs FastKafka application.

    Args:
        num_workers (int): Number of FastKafka instances to run, defaults to the number of CPU cores.
        app (str): Input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.
        kafka_broker (str): Kafka broker, one of the keys of the kafka_brokers dictionary passed in the constructor of FastKafka class.

    Raises:
        typer.Exit: If there is an unexpected internal error.
    """
    try:
        asyncio.run(
            run_fastkafka_server(
                num_workers=num_workers, app=app, kafka_broker=kafka_broker
            )
        )
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: run [OPTIONS] APP                                                                                          
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --num-workers               INTEGER  Number of FastKafka instances to run, defaults to number of CPU cores.     │
│                                      [default: 16]                                                              │
│ --kafka-broker              TEXT     kafka_broker, one of the keys of the kafka_brokers dictionary passed in    │
│                                      the constructor of FastaKafka class.                                       │
│                                      [default: localhost]                                                       │
│ --install-completion                 Install completion for the current shell.                                  │
│ --show-completion                    Show completion for the current shell, to copy it or customize the         │
│                                      installation.                                                              │
│ --help                               Show this message and exit.                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# | export


_app.add_typer(_cli_docs._docs_app, name="docs")

In [ ]:
# | notest

! nbdev_export

In [ ]:
result = runner.invoke(_app, ["docs", "install_deps", "--help"])

                                                                                                                   
 Usage: root docs install_deps [OPTIONS]                                                                           
                                                                                                                   

Installs dependencies for FastKafka documentation generation

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["docs", "install_deps"])
assert result.exit_code == 0

[INFO] fastkafka._components.docs_dependencies: AsyncAPI generator installed


In [ ]:
result = runner.invoke(_app, ["docs", "generate", "--help"])

                                                                                                                   
 Usage: root docs generate [OPTIONS] APP                                                                           
                                                                                                                   

Generates documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created; default is current directory      │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as import_str:
    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

    result = runner.invoke(_app, ["docs", "generate", import_str])
    typer.echo(result.output)
    assert result.exit_code == 0

[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmp6q3pu5j7/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmp6q3pu5j7/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmp6q3pu5j7/asyncapi/docs.






In [ ]:
result = runner.invoke(_app, ["docs", "serve", "--help"])

                                                                                                                   
 Usage: root docs serve [OPTIONS] APP                                                                              
                                                                                                                   

Generates and serves documentation for a FastKafka application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT     root path under which documentation will be created; default is current directory   │
│ --bind             TEXT     Some info [default: 127.0.0.1]                                                      │
│ --port             INTEGER  Some info [default: 8000]                                                           │
│ --help                      Show this message and exit.                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
        "fastkafka",
        "docs",
        "serve",
        "--port=48000",
        app,
        stdout=asyncio.subprocess.PIPE,
    )
    time.sleep(30)
    await terminate_asyncio_process(proc)
    outputs, errs = await proc.communicate()
    expected_returncode = 15 if platform.system() == "Windows" else 0
    assert proc.returncode == expected_returncode, outputs.decode("utf-8")

/home/kumaran/.local/lib/python3.11/site-packages/pydantic/_internal/_config.py:257: UserWarning: Valid config keys have changed in V2:
* 'json_encoders' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
# | export


_app.add_typer(_cli_testing._testing_app, name="testing")

In [ ]:
result = runner.invoke(_app, ["testing", "install_deps", "--help"])

                                                                                                                   
 Usage: root testing install_deps [OPTIONS]                                                                        
                                                                                                                   

Installs dependencies for FastKafka app testing

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["testing", "install_deps"])
assert result.exit_code == 0